# 1. <a id='toc1_'></a>[Import packages](#toc0_)

First, we import the necessary packages for this exercise.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature

%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [ ]:
# Change to directory where data is stored
data_dir = "/kyukon/data/gent/courses/2024/computphys_C004504/input"

**Table of contents**<a id='toc0_'></a>    
- 1. [Import packages](#toc1_)    
- 2. [Open simulation data](#toc2_)    
  - 2.1. [Solutions:](#toc2_1_)    
- 3. [Plot the accumulated precipitation](#toc3_)    
  - 3.1. [Solutions:](#toc3_1_)    
- 4. [Compare with observational gridded data](#toc4_)    
  - 4.1. [Solutions:](#toc4_1_)    
- 5. [Compare with observational station data](#toc5_)    
  - 5.1. [Solutions:](#toc5_1_)    
- 6. [Work with ensemble data](#toc6_)    
  - 6.1. [Solutions:](#toc6_1_)    
- 7. [Extra. RMI warning levels](#toc7_)    
  - 7.1. [Solutions:](#toc7_1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 2. <a id='toc2_'></a>[Open simulation data](#toc0_)

In this exercise we will take a look at the precipitation event from July 2021 which caused massive flooding in the Vesder basin. Some more information about this event can be found [here](https://www.vrt.be/vrtnws/nl/2022/07/08/tijdslijn/) (in Dutch) and [here](https://en.wikipedia.org/wiki/2021_European_floods) (in English). We have simulated this case in several different configurations. All the output data can be found in the 'data' folder: `/data/gent/courses/2024/computphys_C004504/input`. This data is stored in netCDF-files, which can be easily read as xarray Datasets (see introductory notebook on xarray). Each of the netCDF-files has a structured name `<configuration>_<starttime>_<nhours>_<variable>_<tstart>_<tstop>_<tstep>_regridded.nc` where the different elements have the following meaning:
1. `<configuration>`: This refers to the specific set-up of the model. For some experiments, we can change certain parameters in the model and investigate their impact. For this exercise, however, we will only use the data from the 'baseline' simulation.
2. `<starttime>`: This refers to the start time of the simulation. By varying the start time, we can create an ensemble of different simulations of the same event (see below). The start time is written in the format of `<year><month><day><hour>`.
3. `<nhours>`: This refers to the number of hours that the simulation has run.
4. `<variable>`: This refers to the variable saved in this file. We use the standard abbreviations from the Climate and Forecast (CF) Metadata Conventions. The ones you will mostly need in these exercises are:
    - *pr*: Precipitation
    - *tas*: Near-Surface Air Temperature
5. `<tstart>`: This refers to the first time step of the data in the file. This is written in the format of `<year>-<month>-<day>T<hour>`.
6. `<tstart>`: This refers to the last time step of the data in the file. This is written in the format of `<year>-<month>-<day>T<hour>`.
7. `<tstep>`: This refers to the time between subsequent time steps in the data, expressed in seconds. You will see that this is 3600 s for all files (so all data have an hourly resolution).
8. `regridded`: This refers to the fact that the data has been regridded from the orginal coordinate reference system (CRS) to a latitude-longitude reference system. This makes the data easier to work with.
9. `.nc`: This extension denotes the fact that this is a netCDF-file.

Let's start by opening a file and loading in the data! Load in the precipitation data corresponding to the baseline simulation which started at midnight of 1 July 2021 and ran for 384 hours. You can use the xarray function `open_dataset` ([documentation](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html)). Load in this dataset as the Python variable `ds` and inspect its contents by printing `ds` or simply running a cell which ends with `ds`. The following questions/tasks will help you along:
- What are the dimensions of the data? Where can you find information about these dimensions?
- First consider the spatial dimensions. What are the units? What is the resolution? Calculate the resolution in (kilo)metres. Use the fact that the radius of the Earth is 6371 km.
- Next, consider the attributes of the variable `pr`. What are its units? Usually, precipitation flux is expressed in mm/h. Convert the units in the file (if necessary) to $\text{mm}/\text{h}$. Use the fact that water has a density of 1000 $\text{kg}/\text{m}^3$ in your calculations.
- Lastly, in the attributes of the variable `pr`, you can find `cell_methods = time: mean`. This signifies that the variable does not represent an instantaneous value at a certain moment in time, but rather provides an average value over a certain time interval. The bounds of this time interval are given in the coordinate `time_bnds`. So, every value of the coordinate `time` has two corresponding values in the coordinate `time_bnds`: the start and end time of the interval over which the average is taken. Compare the values in `time_bnds` to those in `time`. For a `time` value at moment $x$, which interval corresponds to this? What is the advantage of providing the average precipitation flux, instead of the instantaneous value?

## 2.1. <a id='toc2_1_'></a>[Solutions:](#toc0_)
- Dimensions are (time, latitude, longitude).
- The units are degrees. The resolution is 0.07° for longitude and 0.045° for latitude. We can convert this to kilometres at a certain latitude $\phi$, with $R$ the radius of the Earth:
    - $\Delta x = \left( \frac{\Delta \text{lon}}{360\degree} \right)$ * (circumference of circle of constant latitude) = $\left(\frac{\Delta \text{lon}}{360\degree}\right) (2 \pi R \cos \phi)$
    - $\Delta y = \left( \frac{\Delta \text{lat}}{360\degree} \right)$ * (circumference of circle of constant longitude) = $\left(\frac{\Delta \text{lat}}{360\degree}\right) (2 \pi R)$

    We can see that while $\Delta x$ depends on the latitude, $\Delta y$ does not. Filling in the numbers and using a latitude of $50.5\degree$ we find: $\Delta x = 4.95$ km and $\Delta y = 5.00$ km
- We convert $\text{kg} / (\text{m}^2 \cdot \text{s})$ to $\text{mm}/\text{h}$:
    $$1 \frac{\text{kg}}{\text{m}^2 \cdot \text{s}} = \frac{10^{-3} \text{m}^3}{\text{m}^2 \cdot \frac{1}{3600} \text{h}} = 3600 \frac{10^{-3} \text{m}^3}{\text{m}^2 \text{h}} = 3600 \frac{10^{-3} \text{m}}{ \text{h}} = 3600 \frac{\text{mm}}{ \text{h}}$$
    So a precipitation flux expressed in $\text{kg} / (\text{m}^2 \cdot \text{s})$ needs to be multiplied by 3600 to convert it to $\text{mm}/\text{h}$.
- The value at time $x$ corresponds to the average over the interval $[x - 1 h, x]$. Providing the hourly average precipitation flux enables us to exactly calculate the total amount of precipitation, which would not be possible if only the instantaneous flux were provided hourly.

In [ ]:
ds = xr.open_dataset(
    f"{data_dir}/baseline_2021070100_384_pr_2021-07-01T01_2021-07-17T00_3600_regridded.nc",
    engine="netcdf4",
    chunks="auto",
)
ds

In [ ]:
dlon = 0.07
dlat = 0.045
R = 6371
lat = 50.5

dx = (dlon / 360) * (2 * np.pi * R * np.cos(lat * np.pi / 180))
dy = (dlat / 360) * (2 * np.pi * R)

print(f"dx = {dx:.2f} km ; dx = {dy:.2f} km")

In [ ]:
def convert_to_mm_h(ds):
    if "pr" in ds:
        ds.pr.values = ds.pr.values * 3600
        ds.pr.attrs["units"] = "mm/h"

    return ds


ds = convert_to_mm_h(ds)
ds

In [ ]:
idx = 0
time = ds.time[idx]
time_bnds = ds.time_bnds[idx]

print(f"time : {time.values}")
print(f"time bounds : {time_bnds.values}")

# 3. <a id='toc3_'></a>[Plot the accumulated precipitation](#toc0_)

Let's now focus on the period of the Vesder event. In this part you will make a plot of the accumulated precipitation (expressed in $\text{mm}$) over the period between 13 July 2021 06:00 and 15 July 2021 06:00. This is the period during which most of the precipitation fell during the event. You can use the following code to make your plot (but of course you will have to make changes!). This code shows you how to plot the data on a geographical map with coastlines and borders.

- Calculate the accumulated precipitation (in $\text{mm}$) for the period between 13 July 2021 06:00 and 15 July 2021 06:00.
- Make a plot of this quantity and inspect the plot. 
- Where did most of the precipitation fall? What is the maximum value of accumulated precipitation during this period? Compare these values with normal climatological values. For the Dutch (and French) speakers, you can use the following webpage of the RMI (KMI): [klimaatatlas](https://www.meteo.be/nl/klimaat/klimaat-van-belgie/klimaatatlas).

In [ ]:
# We define some dummy data with the same dimensions to plot. The data is filled with ones.
# For your plot, you of course have to use the simulation data!
da_dummy = xr.DataArray(
    data=np.ones((70, 75)),
    dims=("lat", "lon"),
    coords={"lon": np.arange(2.0, 7.2, 0.07), "lat": np.arange(49.0, 52.11, 0.045)},
)

# Change the values of these levels to see what they define
levels = np.arange(0.85, 1.16, 0.1)

# Create a figure. The projection keyword tells matplotlib (and Cartopy) that we are plotting geographical data. The ccrs.PlateCarree() then denotes that the data is in lat-lon format.
fig, ax = plt.subplots(
    layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()}
)
dummy_plot = da_dummy.plot(ax=ax, x="lon", y="lat", levels=levels, add_colorbar=False)

# Add a title to your plot
ax.set_title("Title of plot")

# This adds coastlines and borders to the plot.
ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")

# This aspect-ratio controls the ratio of the x- and y-axes. Try and find a value which provides a nice picture of our country. (Use the information from above about the spatial resolution and the values in kilometres)
aspect_ratio = 1.0
ax.set_aspect(aspect_ratio)

# Add a color bar and a title to the figure
fig.colorbar(dummy_plot, label="Colorbar label")
fig.suptitle("Title of figure", fontsize="large")

# Show the plot!
plt.show()

## 3.1. <a id='toc3_1_'></a>[Solutions:](#toc0_)

We can see on the figure that in large parts of Wallonia more than 100 mm fell over the course of 48 hours. This is comparable to the normal amount of precipitation for the whole month of July! The most extreme part of the event seems to be located in Germany, where in certain points more than 200 mm fell during this period.

In [ ]:
# Select a period out of the the data with tstart and tstop
tstart = "2021-07-13T07"
tstop = "2021-07-15T06"
ds_period = ds.sel(time=slice(tstart, tstop))

# Select the precipitation out of the data and sum over time
pr_period = ds_period.pr
pr_period_sum = pr_period.sum(dim="time")

# Choose levels for the plot
levels = np.arange(0, 201, 25)

# Plot the data
fig, ax = plt.subplots(
    layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()}
)
pr_plot = pr_period_sum.plot(ax=ax, x="lon", y="lat", levels=levels, add_colorbar=False)
# Set the title to show the maximum plotted value
ax.set_title(f"Maximum value = {pr_period_sum.max().values:.0f} mm")

ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")

# We found previously that while the resolution in degrees is different for the x- and y-axes, the resolution in kilometres is actually the same. To have this represented on the figure, we should therefore modify the aspect ratio accordingly:
dlat = 0.045
dlon = 0.07
aspect_ratio = dlon / dlat
ax.set_aspect(aspect_ratio)

# We give the color bar and the figure appropriate names
fig.colorbar(pr_plot, label="Precipitation depth [mm]")
fig.suptitle(
    f"Accumulated precipitation between {tstart} and {tstop}", fontsize="large"
)

# Show the plot!
plt.show()

# 4. <a id='toc4_'></a>[Compare with observational gridded data](#toc0_)

We will now compare the simulation data with observational data. As observational data, we take the CLIMATE-GRID dataset of the RMI. This is a gridded dataset with a resolution of 5 km. It contains only daily values. For precipitation, the value on a certain day represents accumulated precipitation between 8:00 on the day in question and 8:00 on the following day. It is important to note that the CLIMATE_GRID uses local time (LT), which is CEST (Central European Summer Time) in the summer. There is a difference between summer and winter, because of daylight saving time ("zomertijd"). On the other hand, the time in the simulation is UTC (Universal Time Coordinated). We can convert from CEST to UTC by subtracting 2 hours. So, 8:00 CEST corresponds to 6:00 UTC. (That is why this period has been chosen for the previous parts of this exercise.) 

Follow these steps to compare these observations with the simulation data:
- Load in the CLIMATE-GRID data from the file `CLIMATE_GRID_pr_2021_07_daily_latlon_5km.nc`. Inspect the attributes of the dataset and of the variable `pr`. Convert the units to $\text{mm}/\text{h}$ (if necessary). How can you relate these values to daily accumulated precipitation (expressed in $\text{mm}$)?
- Select the precipitation data corresponding to the period between 13 July 2021 08:00 LT / 06:00 UTC and 15 July 2021 08:00 LT  and plot the accumulated precipitation. Use the same color scheme as for the simulation data. What is the maximum value for the observations? When comparing the figures of the simulation and the observations by eye, what differences do you see?
- Compare the simulations directly with the observations by plotting the bias (difference) in accumulated precipitation over this period. What do you see on this figure?
- Calculate and compare the domain-averaged precipitation values. What can you conclude?

## 4.1. <a id='toc4_1_'></a>[Solutions:](#toc0_)

In [ ]:
filepath = f"{data_dir}/CLIMATE_GRID_pr_2021_07_daily_latlon_5km.nc"
obs_ds = xr.open_dataset(filepath, engine="netcdf4", chunks="auto")
obs_ds = convert_to_mm_h(obs_ds)
obs_ds

In [ ]:
# Select a period out of the the data with tstart and tstop
tstart = "2021-07-13"
tstop = "2021-07-14"
obs_ds_period = obs_ds.sel(time=slice(tstart, tstop))

obs_pr_period = obs_ds_period.pr
obs_pr_period_sum = (
    obs_pr_period.sum("time", skipna=False) * 24
)  # Multiply by 24 to convert the sum from mm/h to mm

# Choose levels for the plot
levels = np.arange(0, 201, 25)

# Plot the data
fig, ax = plt.subplots(
    layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()}
)
pr_plot = obs_pr_period_sum.plot(
    ax=ax, x="lon", y="lat", levels=levels, add_colorbar=False, extend="max"
)
# Set the title to show the maximum plotted value
ax.set_title(f"Maximum value = {obs_pr_period_sum.max().values:.0f} mm")

ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")

# We found previously that while the resolution in degrees is different for the x- and y-axes, the resolution in kilometres is actually the same. To have this represented on the figure, we should therefore modify the aspect ratio accordingly:
dlat = 0.045
dlon = 0.07
aspect_ratio = dlon / dlat
ax.set_aspect(aspect_ratio)

# We give the color bar and the figure appropriate names
fig.colorbar(pr_plot, label="Precipitation depth [mm]")
fig.suptitle(
    f"Accumulated precipitation between {tstart} and {tstop}", fontsize="large"
)

# Show the plot!
plt.show()

In [ ]:
# Select a period out of the simulation data with tstart and tstop
tstart = "2021-07-13T07"
tstop = "2021-07-15T06"
ds_period = ds.sel(time=slice(tstart, tstop))
pr_period = ds_period.pr
pr_period_sum = pr_period.sum(dim="time")

# Select a period out of the observational data with tstart and tstop
tstart = "2021-07-13"
tstop = "2021-07-14"
obs_ds_period = obs_ds.sel(time=slice(tstart, tstop))
obs_pr_period = obs_ds_period.pr
obs_pr_period_sum = obs_pr_period.sum("time", skipna=False) * 24

# Calculate bias/difference
bias_period_sum = pr_period_sum - obs_pr_period_sum

# Choose levels for the plot
levels = [-200, -100, -50, -20, -10, 10, 20, 50, 100, 200]

# Plot the data
fig, ax = plt.subplots(
    layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()}
)
pr_plot = bias_period_sum.plot(
    ax=ax, x="lon", y="lat", levels=levels, add_colorbar=False, extend="both"
)
ax.set_title(
    f"Maximum value = {bias_period_sum.max().values:.0f} mm\nMinimum value = {bias_period_sum.min().values:.0f} mm\nAreal average bias = {bias_period_sum.mean().values:.1f} mm"
)
ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")

dlat = 0.045
dlon = 0.07
aspect_ratio = dlon / dlat
ax.set_aspect(aspect_ratio)

fig.colorbar(pr_plot, label="Precipitation depth [mm]")
fig.suptitle(
    f"Bias in accumulated precipitation\nbetween {tstart} and {tstop}", fontsize="large"
)
plt.show()

From the plotted bias in accumulated precipitation we can see that there are locations with large biases over Belgium. In the Northern part of the country the simulation mostly overestimates the precipitation, while in the South it is underestimated. However, when calculating the areal average of the bias, we can see that the difference between model and observations is relatively small. Therefore, we can conclucde that the model does contain the extreme event, but the event is located too much to the North in the simulation.

# 5. <a id='toc5_'></a>[Compare with observational station data](#toc0_)

We will now take a look at some time series data of the event. For this exercise, you will use station measurements from the five stations which measured the highest precipitation amounts during the Vesder event. These station data are hourly values, contrary to the daily CLIMATE-GRID data. The stations employ the local time (CEST).

- Load in the station data for the five stations from the file `SETHY_pr_2021-07-12T01_2021-07-17T00.nc`. Inspect the structure of this Dataset. Convert (as usual) the precipitation data to $\text{mm}/\text{h}$, if necessary. 
- Make a plot of the locations of the stations on a map. You can use the function `.plot.scatter()`, which you can apply to a Dataset. This function accepts arguments `x` and `y`, which tell the function which Dataset variables/coordinates to use as x- and y-values for the plot. Where are the five stations located? Does it make sense that these stations received the highest precipitation amounts in light of the CLIMATE-GRID data (look at your previous plots)?
- For each of the five stations, plot the time series of hourly precipitation flux for the time period of the event (i.e., precipitation from 13 July 2021 08:00 LT until 15 July 2021 08:00 LT). Also, make a plot of the cumulative precipitation amount over time. What can you conclude when comparing these station observations to the simulated data? Is this in line with previous conclusions? 
- Finally, compare the station measurements to the CLIMATE-GRID data. What do you see?

## 5.1. <a id='toc5_1_'></a>[Solutions:](#toc0_)

In [ ]:
# Open the station data
ds_stations = xr.open_dataset(
    f"{data_dir}/SETHY_pr_2021-07-12T01_2021-07-17T00.nc",
    engine="netcdf4",
    chunks="auto",
)
ds_stations

In [ ]:
# Plot the locations of the stations
fig, ax = plt.subplots(
    layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()}
)
ds_stations.plot.scatter(ax=ax, x="lon", y="lat")
ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
ax.set_aspect(aspect_ratio)
ax.set_xlim(2, 7)
ax.set_ylim(49, 52)
plt.show()

When looking at the plot of acummulated precipitation from CLIMATE-GRID, we see that these stations lie in the region with the highest amount of rainfall during this period.

In [ ]:
# Select simulation and station data for the period of the event
tstart = "2021-07-13T07"
tstop = "2021-07-15T06"
ds_period = ds.sel(time=slice(tstart, tstop))
ds_stations_period = ds_stations.sel(time=slice(tstart, tstop))

# Select observational (CLIMATE-GRID) data for the period of the event
tstart = "2021-07-13"
tstop = "2021-07-14"
obs_ds_period = obs_ds.sel(time=slice(tstart, tstop))

# Key to determine if we plot cumulative values or not
cumulative = True

# Define number of locations
nloc = len(ds_stations.location)

# Initialise figure
fig, axs = plt.subplots(ncols=nloc, figsize=(4.8 * nloc, 4.8), layout="constrained")

# Loop over locations and plot data for each location
for idx in range(nloc):
    location = ds_stations.location[idx]
    lon, lat = ds_stations.lon[idx], ds_stations.lat[idx]

    ax = axs[idx]
    ds_location = ds_period.sel(lon=lon, lat=lat, method="nearest")
    obs_location = obs_ds_period.sel(lon=lon, lat=lat, method="nearest")
    obs_location = obs_location.assign_coords(
        time=obs_ds_period.time + np.timedelta64(30, "h")
    )

    if cumulative:
        ds_stations_period.sel(location=location).pr.cumsum().plot(
            ax=ax, label="station data"
        )
        ds_location.pr.cumsum().plot(ax=ax, label="simulation")
        (obs_location.pr.cumsum() * 24).plot(ax=ax, x="time", label="CLIMATE-GRID")
    else:
        ds_stations_period.sel(location=location).pr.plot(ax=ax, label="station data")
        ds_location.pr.plot(ax=ax, label="simulation")

    station_sum = ds_stations_period.sel(location=location).pr.sum().values
    sim_sum = ds_location.pr.sum().values
    obs_sum = obs_location.pr.sum().values * 24

    ax.set_title(
        f"Location = {location.values}\nobs sum = {station_sum:.0f} mm; sim sum = {sim_sum:.0f} mm;\nCG sum = {obs_sum:.0f} mm"
    )
    ax.legend()

if cumulative:
    fig.suptitle(
        "Time series of accumulated precipitation for simulated, station and CLIMATE-GRID data",
        fontsize="x-large",
    )
else:
    fig.suptitle(
        "Time series of precipitation for simulated and station data",
        fontsize="x-large",
    )

When comparing the simulation with the station measurements, we see that the measured rainfall significantly exceeds the simulated amounts at these locations. The station data also generally give even higher rainfall amounts than the CLIMATE-GRID dataset.

# 6. <a id='toc6_'></a>[Work with ensemble data](#toc0_)

In this exercise we will look not at one simulation but at an ensemble of different simulations. Ensembles consist of several simulations for the same period, which differ slightly in certain aspects. With an ensemble we can gauge the uncertainty of the simulation. Ensembles can be created in several ways, for example: by slightly varying the initial conditions, by slightly changing certain parameters in the model, or simply by using different models. We have created an ensemble by running the baseline simulation, but starting it at different times. This slight change in 'initial conditions' is enough to change the course of the simulation even though the same model set-up and identical boundary conditions are used.

You will now study this ensemble with the help of these steps:
- Load in the precipitation data of ensemble created by running the `baseline` simulation with different start times. These start times vary from 2021-07-01T00 to 2021-07-04T00 with steps of 12 hours. In total, you should have 7 members in your ensemble. Use the `concat`-function to save the data as one Dataset. You can use the `open_mfdataset`-function ([link](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html)) instead of the `open_dataset`, to avoid having to specify the entire path.
- Calculate the bias of the domain-averaged accumulated precipitation during the period between 13 July 2021 08:00 LT / 06:00 UTC and 15 July 2021 08:00 LT / 06:00 UTC for each member of the ensemble compared to the CLIMATE-GRID data. Repeat this exercise but for the two 24-hour periods inside of this 48-hour period. Create a plot to help you visualise these values. What can you conclude?
- Plot the ensemble data for each of the station locations. What do you see?

## 6.1. <a id='toc6_1_'></a>[Solutions:](#toc0_)

In [ ]:
run_name = "baseline"
run_info_list = [
    ("2021070100", 384),
    ("2021070112", 372),
    ("2021070200", 360),
    ("2021070212", 348),
    ("2021070300", 336),
    ("2021070312", 324),
    ("2021070400", 312),
]

ds_ens_list = list()

for run_info in run_info_list:
    rstart, nhours = run_info
    filepath = f"{data_dir}/{run_name}_{rstart}_{nhours}_pr_*.nc"
    ds_ens = xr.open_mfdataset(filepath, engine="netcdf4", chunks="auto")
    ds_ens_list.append(ds_ens)

ds_ens = xr.concat(ds_ens_list, dim="rstart")
ds_ens = convert_to_mm_h(ds_ens)
ds_ens.load()

In [ ]:
tstart = "2021-07-13T07"
tstop = "2021-07-15T06"
ds_ens_period = ds_ens.sel(time=slice(tstart, tstop))
pr_ens_period = ds_ens_period.pr

tstart = "2021-07-13"
tstop = "2021-07-14"
obs_ds_period = obs_ds.sel(time=slice(tstart, tstop))
obs_pr_period = obs_ds_period.pr

fig, ax = plt.subplots(layout="constrained")

obs_mask = ~obs_ds_period.isnull().any("time")
pr_ens_period_cumsum = (
    pr_ens_period.cumsum(dim="time").where(obs_mask.pr).mean(["lat", "lon"])
)
pr_ens_period_cumsum.plot(ax=ax, x="time", hue="rstart")
obs_pr_period_cumsum = (
    obs_pr_period.mean(["lat", "lon"])
    .cumsum(dim="time")
    .assign_coords(time=obs_ds_period.time + np.timedelta64(30, "h"))
    * 24
)
obs_pr_period_cumsum.plot.scatter(ax=ax, x="time", color="black")

We can find the values for the bias as follows:

In [ ]:
bias_ens_period_cumsum = pr_ens_period_cumsum - obs_pr_period_cumsum
bias_ens_period_cumsum

The left column represents the bias for the accumulated precipitation of the first 24 hours and the right column for the entire 48-hour period. The bias of the second 24 hours can be found by subtracting the two follows from each other:

In [ ]:
bias_ens_period_cumsum.diff(dim="time")

While the 48-hour accumulated precipitation amounts averaged over Belgium are well represented by the ensemble, albeit slightly overestimated, this is not the case for the two 24-hour periods. For the first 24 hours, the precipitation is overestimated by the ensemble (positive bias), which is compensated by an underestimation during the second 24 hours (negative bias).

In [ ]:
# Select the simulation and station data for the period of the event
tstart = "2021-07-13T07"
tstop = "2021-07-15T06"
ds_ens_period = ds_ens.sel(time=slice(tstart, tstop))
ds_stations_period = ds_stations.sel(time=slice(tstart, tstop))

# Key to determine if we plot cumulative values or not
cumulative = True

# Define number of locations
nloc = len(ds_stations.location)

# Initialise figure
fig, axs = plt.subplots(ncols=nloc, figsize=(4.8 * nloc, 4.8), layout="constrained")

# Loop over locations and plot data for each location
for idx in range(nloc):
    location = ds_stations.location[idx]
    lon, lat = ds_stations.lon[idx], ds_stations.lat[idx]

    ax = axs[idx]
    ds_ens_location = ds_ens_period.sel(lon=lon, lat=lat, method="nearest")

    if cumulative:
        ds_stations_period.sel(location=location).pr.cumsum(dim="time").plot(
            ax=ax, label="station data"
        )
        ds_ens_location.pr.cumsum(dim="time").plot(ax=ax, hue="rstart")
    else:
        ds_stations_period.sel(location=location).pr.plot(ax=ax, label="station data")
        ds_ens_location.pr.plot(ax=ax, hue="rstart")

    station_sum = ds_stations_period.sel(location=location).pr.sum().values
    sim_sum = ds_ens_location.pr.sum(dim="time").mean(dim="rstart").values

    ax.set_title(
        f"Location = {location.values}\nobs sum = {station_sum:.0f} mm; sim sum = {sim_sum:.0f} mm"
    )
    ax.legend()

if cumulative:
    fig.suptitle(
        "Time series of accumulated precipitation for simulated and station data",
        fontsize="x-large",
    )
else:
    fig.suptitle(
        "Time series of precipitation for simulated and station data",
        fontsize="x-large",
    )

All simulations of the ensemble underestime the observed rainfall amount during this period. At some locations the spread of the ensemble is quite large.

# 7. <a id='toc7_'></a>[Extra. RMI warning levels](#toc0_)

We will now consider the severity of the event from a different angle. The Royal Meteorological Institute of Belgium (RMI or KMI in Dutch) issues color-coded warning levels to warn the public about an upcoming extreme events. The colors for these warnings are yellow, orange and red. Warnings exist for all types of events, such as heatwaves, wind and thunder storms, but also for precipitation/rain! Certain thresholds are used by the institute to assign a color to an event. Information about these thresholds can be found on [warnings](https://www.meteo.be/nl/weer/waarschuwingen/weerflashes) (you can change the language in the upper right corner of the page).

Investigate the simulated precipitation during the period between 13 July 2021 06:00 UTC and 15 July 2021 06:00 UTC and determine at which points a yellow/orange/red warning for rain would be issued at any time during this period based on this simulation data. The following functions will probably be helpful for this exercise:
- the `rolling` function: [documentation](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.rolling.html#xarray.DataArray.rolling)
- the `where` function: [documentation](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.where.html#xarray.Dataset.where)

Based on this simulation data, for which parts of the country would a red warning be issued?

## 7.1. <a id='toc7_1_'></a>[Solutions:](#toc0_)

In [ ]:
def mask_pr(pr, color):
    thresholds_dict = {
        "yellow": [(1, 20, 30), (6, 20, 40), (24, 25, 50)],
        "orange": [(1, 30, 50), (6, 40, 60), (24, 50, 100)],
        "red": [(1, 50, np.inf), (6, 60, np.inf), (24, 100, np.inf)],
    }

    assert color in thresholds_dict.keys(), f"Color {color} not accepted"
    thresholds_list = thresholds_dict[color]

    mask_list = list()

    for idx in range(len(thresholds_list)):
        hours, min_thresh, max_thresh = thresholds_list[idx]

        pr_rolling_sum = pr.rolling(time=hours).sum()
        min_mask = min_thresh < pr_rolling_sum
        max_mask = pr_rolling_sum <= max_thresh
        mask = (min_mask & max_mask).any("time")

        mask_list.append(mask.assign_coords(idx=idx))

    mask = xr.concat(mask_list, dim="idx").any("idx")
    return pr.where(mask)

In [ ]:
tstart = "2021-07-13T07"
tstop = "2021-07-15T06"
ds_period = ds.sel(time=slice(tstart, tstop))
pr_period = ds_period.pr

fig, axs = plt.subplots(
    ncols=3,
    figsize=(19.2, 4.8),
    layout="constrained",
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for idx in range(3):
    color = ["yellow", "orange", "red"][idx]
    pr_period_masked = mask_pr(pr_period, color)
    pr_period_masked_sum = pr_period_masked.sum("time", skipna=False)

    ax = axs[idx]
    pr_period_masked_sum.plot(
        ax=ax,
        levels=np.arange(0, 201, 25),
        extend="max",
        cbar_kwargs={"label": "Precipitation depth [mm]"},
    )
    ax.set_title(f"Warning level {color}")
    ax.coastlines(alpha=0.7)
    ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
    ax.set_aspect(aspect_ratio)